# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [16]:
# Standard lib imports
import os
from dotenv import load_dotenv
from tavily import TavilyClient
from rich import print as rprint
from typing import Optional 
# Custom lib imports
from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager

In [2]:

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL")

In [3]:
# access the vector store created in Part 1
manager = VectorStoreManager(
    openai_api_key=OPENAI_API_KEY,
    chroma_path="chromadb",)

vector_store = manager.get_store("udaplay")

In [12]:
# test if store has loaded well
if vector_store is None:
    print("❌ Store not found! Did you use the correct name ('game_knowledge')?")
    print("Existing collections:", [c.name for c in manager.chroma_client.list_collections()])
else:
    # Access the underlying Chroma collection to count documents
    count = vector_store._collection.count()
    print(f"✅ VectorStore loaded successfully!")
    print(f"📊 Document Count: {count}")
    
    # 3. Sanity check: Fetch one item to ensure data is readable
    if count > 0:
        sample = vector_store.query(query_texts="Which game has action?", n_results=1)
        print("\n Sample Document ID:", sample['ids'][0])
        rprint("\n Content Snippet:", sample['documents'] )

✅ VectorStore loaded successfully!
📊 Document Count: 15

 Sample Document ID: ['002']


Content Snippet:
[
    [
        "[PlayStation 2] Grand Theft Auto: San Andreas - Genre: Action-adventure (2004 by Rockstar Games) - An 
expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."
    ]
]

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [32]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# collection = chroma_client.get_collection("udaplay")
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

@tool

def retrieve_game(query: str, results: Optional [int] = 3) -> str:
    """
    Search for video game information in the internal knowledge base.
    Args:
        query (str): A question about the game industry.
        results (int, optional): Number of top results to retrieve. Defaults to 3.
    Returns:
        str: Formatted string containing game information from the knowledge base
 
       """
    # 1. Query with specific include parameters to ensure we get metadata and distances
    results = vector_store.query(
        query_texts=[query], 
        n_results=results,
    )
    
    # 2. Unpack the batch results (we only sent one query, so we take index 0)
    docs = results.get('documents', [[]])[0]
    metas = results.get('metadatas', [[]])[0]
    distances = results.get('distances', [[]])[0]
    
    if not docs:
        return "No relevant game information found in the knowledge base."

    # 3. Format the output nicely
    formatted_output = []
    for doc, meta, distance in zip(docs, metas, distances):
        # Create a structured block for each game found
        entry = (
            f"### Game: {meta.get('Name', 'Unknown Title')}\n"
            f"- **Platform**: {meta.get('Platform', 'N/A')}\n"
            f"- **Year**: {meta.get('YearOfRelease', 'N/A')}\n"
            f"- **Publisher**: {meta.get('Publisher', 'N/A')}\n"
            f"- **Details**: {doc}\n"
            f"- **Relevance Score**: {distance:.4f}\n"
        )
        formatted_output.append(entry)

    return "\n".join(formatted_output)

# Test the tool
rprint(retrieve_game("Which game has action gameplay and was released on PlayStation 2?"))

### Game: Grand Theft Auto: San Andreas
- **Platform**: PlayStation 2
- **Year**: 2004
- **Publisher**: Rockstar Games
- **Details**: [PlayStation 2] Grand Theft Auto: San Andreas - Genre: Action-adventure (2004 by Rockstar Games) - 
An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson.
- **Relevance Score**: 0.1765

### Game: Marvel's Spider-Man 2
- **Platform**: PlayStation 5
- **Year**: 2023
- **Publisher**: Sony Interactive Entertainment
- **Details**: [PlayStation 5] Marvel's Spider-Man 2 - Genre: Action-adventure (2023 by Sony Interactive 
Entertainment) - The sequel to the acclaimed Spider-Man game, featuring both Peter Parker and Miles Morales as 
playable characters.
- **Relevance Score**: 0.1796

### Game: Gran Turismo
- **Platform**: PlayStation 1
- **Year**: 1997
- **Publisher**: Sony Computer Entertainment
- **Details**: [PlayStation 1] Gran Turismo - Genre: Racing (1997 by Sony Computer Entertainment) - A realistic 
racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.
- **Relevance Score**: 0.1873

#### Evaluate Retrieval Tool

In [ ]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

@tool
def evaluate_retrieval(question: str, retrieved_docs: str) -> str:
    """
    Evaluate the usability of retrieved documents to answer a user's question.
    Args:
        question (str): The original question from the user.
        retrieved_docs (str): Retrieved documents most similar to the user query in the Vector Database.
    Returns:
        str: Evaluation report indicating usefulness and description.
        - useful: whether the documents are useful to answer the question
        - description: description about the evaluation result
    """
    # Initialize LLM for evaluation
    llm = LLM(
        model="gpt-4o-mini",
        api_key=OPENAI_API_KEY,
        temperature=0.3
    )
    
    # Create the evaluation prompt
    evaluation_prompt = (
        f"Your task is to evaluate if the following documents are sufficient to respond to the question.\n\n"
        f"Question: {question}\n\n"
        f"Retrieved Documents:\n{retrieved_docs}\n\n"
        f"Please provide an evaluation report that includes:\n"
        f"- useful: whether the documents are useful to answer the question (True/False)\n"
        f"- description: detailed explanation of your evaluation."
    )
    
    # Get evaluation from LLM
    evaluation_response = llm.invoke([UserMessage(content=evaluation_prompt)])
    
    return evaluation_response.content

# Test evaluate_retrieval tool
rprint(evaluate_retrieval(
    "Which game has action gameplay and was released on PlayStation 2?",
    retrieve_game("Which game has action gameplay and was released on PlayStation 2?")
))

### Evaluation Report

- **Useful**: True

- **Description**: The question asks for a game that has action gameplay and was released on PlayStation 2. Among 
the retrieved documents, "Grand Theft Auto: San Andreas" is the only game that meets both criteria. It is 
explicitly stated to be an action-adventure game and was released on the PlayStation 2 in 2004. 

The other two documents, "Marvel's Spider-Man 2" and "Gran Turismo," do not meet the requirements of the question. 
"Marvel's Spider-Man 2" is for PlayStation 5 and was released in 2023, thus not applicable to the PlayStation 2. 
"Gran Turismo," while a well-known game, is for PlayStation 1 and falls under the racing genre, which does not 
align with the action gameplay requested.

Therefore, despite the presence of irrelevant documents, the inclusion of "Grand Theft Auto: San Andreas" makes the
retrieved documents sufficient to answer the question.

#### Game Web Search Tool

In [33]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)    

@tool
def game_web_search(query: str, num_results: Optional[int] = 3) -> str:
    """
    Search the web for current video game information.
    Args:
        query (str): A question about the game industry.
    Returns:
        str: Formatted string containing web search results.
    """
    try:
        response = tavily_client.search(
            query, 
            search_depth="basic",
            max_results=num_results
            )
        
        formatted_output = []
        total_results = len(response.get('results')) # since we might get less than requested
        
        rprint(f"Found {total_results} results for query: '{query}'\n")
        for result in response.get('results', [])[:total_results]:
            title = result.get('title', 'No Title')
            url = result.get('url', '#')
            content = result.get('content', 'No content available.')
            
            # Format as a clear source block
            entry = (
                f"### Web Result: {title}\n"
                f"- **Source**: {url}\n"
                f"- **Summary**: {content}\n"
            )
            formatted_output.append(entry)
            
        return "\n".join(formatted_output)

    except Exception as e:
        return f"Error performing web search: {str(e)}"

# Test game_web_search tool
rprint(game_web_search("Which game has action gameplay and was released on PlayStation 2?"))

Found 3 results for query: 'Which game has action gameplay and was released on PlayStation 2?'

### Web Result: Good action games - PlayStation 2 - GameFAQs
- **Source**: https://gamefaqs.gamespot.com/boards/915821-playstation-2/67098409
- **Summary**: Manhunt, it's an amazing and original stealth game. The Getaway Black Monday, it has an interesting 
storyline, for a game. Law & Order: Criminal

### Web Result: List of Action & Adv games for PlayStation 2
- **Source**: https://www.estarland.com/platforms/Playstation2/9?category=AA
- **Summary**: 187: Ride or Die. PlayStation 2. Used. $18.36. VIEW DETAILS. 24: The Game. PlayStation 2. Used. 
$7.16. VIEW DETAILS. 25 To

### Web Result: The PS2 was home to some of the best action RPGS! Here ...
- **Source**: https://www.facebook.com/groups/558977328068750/posts/1004877000145445/
- **Summary**: Over the top cutscenes, insane gameplay and an iconic soundtrack. Virtua Fighter 4: Evolution; 
flawless in almost every way imaginable for a

### Agent

In [34]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

agent = Agent(
    model_name="gpt-4o-mini",
    instructions=(
        "You are an Agentic video game expert that can intelligently decide which tools to use"
        "to answer user questions about video games. Reason about the response, change the query and call "
        "the tool again if needed in order to get better results. Always explain your reasoning and provide " 
        "comprehensive answers"
        ),
    tools=[retrieve_game, game_web_search, evaluate_retrieval],
)


In [46]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
questions = [
    "When was Pokémon Gold and Silver released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for Playstation 5?"
]
for question in questions:
    rprint(f"\n\n### ❓ Question: {question}\n")
    response = agent.invoke(question)
    rprint(f"### 🤖 Agent Response:\n{response.get_final_state()['messages'][-1].content}\n")



### ❓ Question: When was Pokémon Gold and Silver released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


### 🤖 Agent Response:
Pokémon Gold and Silver were released in **1999** for the Game Boy Color. These games are part of the second 
generation of Pokémon, introducing new Pokémon species, regions, and gameplay mechanics. If you have any more 
questions or need further information, feel free to ask!

### ❓ Question: Which one was the first 3D platformer Mario game?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


### 🤖 Agent Response:
The first 3D platformer Mario game is **Super Mario 64**, released in **1996** for the Nintendo 64. This game was 
groundbreaking and set new standards for the platforming genre by introducing players to a fully realized 3D 
environment. In Super Mario 64, players control Mario as he navigates through various worlds to rescue Princess 
Peach, utilizing innovative gameplay mechanics such as open-world exploration and various power-ups. If you have 
more questions about Mario games or anything else, feel free to ask!

### ❓ Question: Was Mortal Kombat X released for Playstation 5?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


### 🤖 Agent Response:
Mortal Kombat X was not specifically released for PlayStation 5; it originally launched in 2015 for PlayStation 4, 
Xbox One, and Microsoft Windows. However, it is playable on the PS5 through backward compatibility. Players may 
need to ensure their PS5 is updated to the latest system software for optimal performance.

If you have any more questions or need further information about Mortal Kombat X or any other games, feel free to 
ask!

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

Add long-term memory

Create the agent as a defined state machine